In [1]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
%load_ext cython
%reload_ext cython
%load_ext autoreload

In [2]:
%autoreload
import sys
sys.path.append("../spykesim")
import spykesim.editsim as sp

In [3]:
resolution = 5000
c = (0.322+0.05j)
# sp.calc_julia(resolution, c)

In [4]:
A = np.random.random(1000)
B = np.random.random(1000)
C = np.random.random(1000)

In [4]:
sp.test_parallel()

AttributeError: module 'spykesim.editsim' has no attribute 'test_parallel'

In [5]:
sp.parallel_loop(A, B, C)

AttributeError: module 'spykesim.editsim' has no attribute 'parallel_loop'

In [ ]:
sp.d

In [11]:
nrow = 100
mat1 = np.random.randint(0, 10, size = nrow ** 2).reshape(nrow, nrow).astype(np.double)
mat2 = np.random.randint(0, 10, size = nrow ** 2).reshape(nrow, nrow).astype(np.double)

NameError: name 'np' is not defined

# Simple editsim

## Simple

In [4]:
def pysimpleeditsim(mat1, mat2):
    nrow = mat1.shape[1]
    ncol = mat2.shape[1]
    dp_table = np.zeros((nrow+1, ncol+1))
    for col1 in range(nrow):
        for col2 in range(ncol):
            match = np.dot(mat1[:, col1], mat2[:, col2])
            dp_table[col1 + 1, col2 + 1] = np.max([
                dp_table[col1, col2 + 1],
                dp_table[col1 + 1, col2],
                dp_table[col1, col2] + match])
    return dp_table[-1, -1]

In [5]:
%timeit pysimpleeditsim(mat1, mat2)

67.2 ms ± 3.81 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%timeit csimpleeditsim(mat1, mat2)

893 µs ± 15.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
pysimpleeditsim(mat1, mat2) == csimpleeditsim(mat1, mat2)

True

## With bp

In [9]:
def pysimpleeditsim_withbp(mat1, mat2):
    nrow = mat1.shape[1]
    ncol = mat2.shape[1]
    dp_table = np.zeros((nrow+1, ncol+1))
    bp_table = np.ones_like(dp_table) * (-1)
    for col1 in range(nrow):
        for col2 in range(ncol):
            match = np.dot(mat1[:, col1], mat2[:, col2])
            choices = [
                dp_table[col1, col2 + 1],
                dp_table[col1 + 1, col2],
                dp_table[col1, col2] + match
            ]
            bp_table[col1 + 1, col2 + 1] = np.argmax(choices)
            dp_table[col1 + 1, col2 + 1] = np.max(choices)            
    return dp_table[-1, -1], bp_table

In [10]:
%timeit pysimpleeditsim_withbp(mat1, mat2)

124 ms ± 2.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%timeit csimpleeditsim_withbp(mat1, mat2)

1.05 ms ± 15 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
pysimpleeditsim_withbp(mat1, mat2)[0], csimpleeditsim_withbp(mat1, mat2)[0]

(205628.0, 205628.0)

In [16]:
(np.sum(pysimpleeditsim_withbp(mat1, mat2)[1] == csimpleeditsim_withbp(mat1, mat2)[1])) == (101*101)

True

## align

# local_exp editsim